# Kaggle: Intermediate Machine Learning

This notebook is an exercise in the [Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning) course. The tutorial can be referenced [here](https://www.kaggle.com/code/alexisbcook/introduction/tutorial). The aim of this notebook is to provide a preliminary analysis into a Random Forest Regressor for the [Housing Prices Competition on Kaggle](https://www.kaggle.com/competitions/home-data-for-ml-course/overview).

## Importing the Necessary Packages and Loading the Datasets

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [1]:
# Reading the data
X_full = pd.read_csv('home-data-for-ml-course/train.csv', index_col='Id') 
X_test_full = pd.read_csv('home-data-for-ml-course/test.csv', index_col='Id')

## Preparing the Data

In [2]:
# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

## Validation Set Split

A validation set is then obtained from the training set using the industry standard 80/20 split. 

In [3]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size = 0.8, test_size = 0.2, random_state = 42)

## Cardinality

It is the number of unique values in a column. The below code selects the categorical columns with relatively low cardinality.

In [4]:
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [5]:
# Keeing only selected columns
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [6]:
X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
255,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,294,250,0,0,0,0,0,0,6,2010
1067,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,380,0,40,0,0,0,0,0,5,2009
639,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Feedr,Norm,...,0,328,0,164,0,0,0,0,5,2008
800,RL,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,Feedr,Norm,...,240,0,0,264,0,0,0,0,6,2007
381,RL,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,308,0,0,242,0,0,0,0,5,2010


## Preprocessing and Training the Model

In [8]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy = 'constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [9]:
# Define model
model = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [10]:
# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps = [('preprocessor', preprocessor),
                        ('model', model)
                       ])

In [11]:
# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBa...
                                                 

In [12]:
# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

In [13]:
print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17678.294143835617


The code obtains a value around 17678 for the mean absolute error (MAE). The next step amends the code to perform even better.

## Improving the Performance

In [14]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy = 'constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'constant')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [15]:
# Defining the model
model = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [16]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps = [('preprocessor', preprocessor),
                                ('model', model)
                               ])

In [17]:
# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBa...
                                                 

In [18]:
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

In [19]:
# Evaluating the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 17655.71462328767


## Generating Test Predictions

In [20]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test)

In [21]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('pipelineSubmission.csv', index=False)